<a href="https://colab.research.google.com/github/Nakul24-1/Personality-Analysis/blob/master/Copy_of_NEU_trait_RoBERTa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
!pip install transformers==3.5.1
!pip install tensorflow

# Recommended tensorflow version is <= 2.1.0, otherwise F1 score function breaks

from transformers import RobertaModel
from transformers import RobertaTokenizer
import os
import tensorflow as tf
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
import tensorflow_datasets as tfds


     |████████████████████████████████| 1.3MB 7.7MB/s 
     |████████████████████████████████| 890kB 28.0MB/s 
     |████████████████████████████████| 2.9MB 51.1MB/s 
     |████████████████████████████████| 1.1MB 48.3MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=52a0a376b1183d0794142ea8029187db2deac812ecc5176c6209ce081aa14f9b
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


ImportError: ignored

In [ ]:
import torch
torch.version

<module 'torch.version' from '/usr/local/lib/python3.7/dist-packages/torch/version.py'>

In [ ]:
train_tweets = pd.read_csv('essays2007.csv')
#test_tweets = pd.read_csv('test_tweets.csv')

#test_tweets['label'] = 0

training_sentences, testing_sentences = train_test_split(train_tweets[['text', 'cNEU']],
                                                         test_size=0.2)

roberta_tokenizer = RobertaTokenizer.from_pretrained("roberta-base",use_fast=False)

# can be up to 512 for BERT
max_length = 350

# the recommended batches size for BERT are 32,64 ... however on this dataset we are overfitting quite fast
# and smaller batches work like a regularization.
# You might play with adding another dropout layer instead.(**Have to explore this**)

batch_size = 16


In [ ]:
testing_sentences

,text,cNEU
1499,Psychology is a class that I have always been ...,1
1104,This is probably the weirdest writing assignme...,0
2394,Today was a long day. I have been going none s...,0
229,"Ouch, that hurts. Damn, damn, damn. I really...",0
1033,I am an architecture major. Therefore I do not...,1
...,...,...
2455,I'm finally getting around to typing this stre...,0
1936,"Well, I'm listening to 0 Doors Down. My friend...",0
1352,I'm sitting in the library at 0:00am once agai...,1
265,I guess I won't be looking forward to this wee...,1


In [ ]:
def convert_example_to_feature(review):
    # combine step for tokenization, WordPiece vector mapping and will
    # add also special tokens and truncate reviews longer than our max length
    return roberta_tokenizer.encode_plus(review,
                                 add_special_tokens=True,  # add [CLS], [SEP]
                                 max_length=max_length,  # max length of the text that can go to RoBERTa
                                 pad_to_max_length=True,  # add [PAD] tokens at the end of sentence
                                 return_attention_mask=True,  # add attention mask to not focus on pad tokens
                                 truncation = True,
                                 )

In [ ]:
def map_example_to_dict(input_ids, attention_masks, label):
    return {
      "input_ids": input_ids,
      "attention_mask": attention_masks,
           }, label

def encode_examples(ds, limit=-1):
    # Prepare Input list
    input_ids_list = []
    attention_mask_list = []
    label_list = []


    if (limit > 0):
        ds = ds.take(limit)

    for review, label in tfds.as_numpy(ds):
        bert_input = convert_example_to_feature(review.decode())
        input_ids_list.append((bert_input['input_ids']))
        attention_mask_list.append((bert_input['attention_mask']))
        label_list.append([label])

    return tf.data.Dataset.from_tensor_slices((input_ids_list,
                                               attention_mask_list,
                                               label_list)).map(map_example_to_dict)

training_sentences_modified = tf.data.Dataset.from_tensor_slices((training_sentences['text'],
                                                                  training_sentences['cNEU']))

testing_sentences_modified = tf.data.Dataset.from_tensor_slices((testing_sentences['text'],
                                                                 testing_sentences['cNEU']))

ds_train_encoded = encode_examples(training_sentences_modified).shuffle(10000).batch(batch_size)
ds_test_encoded = encode_examples(testing_sentences_modified).batch(batch_size)




/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2155: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
ds_test_encoded

<BatchDataset shapes: ((None, 350), (None, 350), (None, 1)), types: (tf.int32, tf.int32, tf.int32)>

In [ ]:
learning_rate = 8e-6
number_of_epochs = 8
class ModelMetrics(tf.keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.count_n = 1

    def on_epoch_end(self, batch, logs={}):
        
        os.makedirs('/model' + str(self.count_n),exist_ok=True)
        self.model.save_pretrained('/model' + str(self.count_n)) # this folder address should match with folder we created above
        
        y_val_pred = tf.nn.softmax(self.model.predict(ds_test_encoded))
        y_pred_argmax = tf.math.argmax(y_val_pred, axis=1)
        #testing_copy = testing_sentences.copy()
        #testing_copy['predicted'] = y_pred_argmax
        #f1_s = f1_score(testing_sentences['cAGR'], testing_copy['predicted'])
        #print('\n f1 score is :', f1_s)
        self.count_n += 1

metrics = ModelMetrics()

In [ ]:
metrics

In [ ]:
model = RobertaModel.from_pretrained("roberta-base",return_dict = False)
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate, epsilon=1e-08) #maybe try diff values here
model.summary()
# we do not have one-hot vectors, we can use sparce categorical cross entropy and accuracy
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
#model.add(layers.dropout(0.5))
model.compile(optimizer=optimizer, loss=loss, metrics=[metric])
model.fit(ds_train_encoded, epochs=number_of_epochs,
          validation_data=ds_test_encoded, callbacks=[metrics])



All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

Some layers of TFRobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model: "tf_roberta_for_sequence_classification_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
roberta (TFRobertaMainLayer) multiple                  124055040 
_________________________________________________________________
classifier (TFRobertaClassif multiple                  592130    
Total params: 124,647,170
Trainable params: 124,647,170
Non-trainable params: 0
_________________________________________________________________
Epoch 1/8
124/124 [==============================] - 188s 1s/step - loss: 0.6950 - accuracy: 0.5044 - val_loss: 0.6935 - val_accuracy: 0.4757


ValueError: ignored

In [ ]:
comment_dataframe = pd.read_csv('FYP_Dataset1.csv')
comment_dataframe['label'] = 0
print(comment_dataframe)

In [ ]:
def encode_examples(ds, limit=-1):
    # Prepare Input list
    input_ids_list = []
    attention_mask_list = []
    label_list = []

    if (limit > 0):
        ds = ds.take(limit)

    for review, label in tfds.as_numpy(ds):
        bert_input = convert_example_to_feature(review.decode())
        input_ids_list.append(bert_input['input_ids'])
        attention_mask_list.append(bert_input['attention_mask'])
        label_list.append([label])

    return tf.data.Dataset.from_tensor_slices((input_ids_list,
                                               attention_mask_list,
                                               label_list)).map(map_example_to_dict)

submission_sentences_modified = tf.data.Dataset.from_tensor_slices((comment_dataframe['Essay'],
                                                                            comment_dataframe['pred_val']))
ds_submission_encoded = encode_examples(submission_sentences_modified).batch(batch_size)

import numpy as np

submission_pre = tf.nn.softmax(model.predict(ds_submission_encoded))
submission_pre_argmax = tf.math.argmax(submission_pre, axis=1)
print(np.array(submission_pre[0]))
comment_dataframe['pred_val'] = (np.array(submission_pre[0]))

In [ ]:
newdf=comment_dataframe[['Name','Neurotism','pred_val']]

In [ ]:
from sklearn import preprocessing

x = np.array(newdf['Extraversion']) #returns a numpy array
newdf.iloc[:,1:-1] = newdf.iloc[:,1:-1].apply(lambda x: (x-30)/ (120-30), axis=0)
newdf